###### Imports and Settings

In [1]:
import pandas as pd
import numpy as np
import requests
from functools import reduce
import matplotlib.pyplot as plt
import pickle
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
import sys
sys.path.append("..") # Adds higher directory to python modules path
import geodict
namestocommon = geodict.namestocommon
geotogeoid = geodict.geotogeoid
tofullcensus = geodict.tofullcensus
import sqlite3 as sq

# This notebook outlines the download and formatting process for the University of Wisconsin's Population Health Institute's County Health Rankings for counties places in the GNRC operating region.  

Go to this page: https://www.countyhealthrankings.org/explore-health-rankings/tennessee/data-and-resources  
Download the "(YEAR) Tennessee Data" excel workbook  
If the new version is the same as the 2022 download, then it will contain the following pages:  
+ Introduction, Outcomes & Factors Rankings, Outcomes & Factors SubRankings, Ranked Measure Data, Additional Measure Data, Ranked Measure Sources & Years, Additional Measure Sources & Years  
+ For both "Outcomes & Factors Rankings", and "Outcomes & Factors SubRankings": unmerge headers and keep the FIPS, rename the County to "NAME", delete the State column, and keep only the ranks (we won't use the z-scores), and make sure the column headers are consistent with the outcomes or factors. Ensure the counties are alphabetized and move all of the rankings into one sheet.
  

Save these csvs as they come in the Data Downloads folder of Parent Data Gathering  


8. Safety: Violent Crime Rate per 100,000 population [CHR], Injury Death Rate per 100,000 population [CHR], Homicide Rate per 100,000 population [CHR], Firearm Fatalities Rate per 100,000 population [CHR], Crime Rate per 1,000 population [TBI, 2019], Total Stolen Vehicles in 2019 [TBI,2019] 

In [12]:
#import and drop FIPS on the import as we'll join it with the custom module
data = pd.read_csv('../../Data Downloads/CountyHealthRankings_2022.csv', dtype = str)
data = data.drop(columns = ['FIPS'])
data.head(3)

,NAME,Health Outcomes,Health Factors,Length of Life,Quality of Life,Health Behaviors,Clinical Care,Social & Economic Factors,Physical Environment,# deaths due to COVID-19 during 2020,COVID-19 death rate,Life Expectancy,95% CI - Low,95% CI - High,Life Expectancy (AIAN),Life Expectancy (AIAN) 95% CI - Low,Life Expectancy (AIAN) 95% CI - High,Life Expectancy (Asian),Life Expectancy (Asian) 95% CI - Low,Life Expectancy (Asian) 95% CI - High,Life Expectancy (Black),Life Expectancy (Black) 95% CI - Low,Life Expectancy (Black) 95% CI - High,Life Expectancy (Hispanic),Life Expectancy (Hispanic) 95% CI - Low,Life Expectancy (Hispanic) 95% CI - High,Life Expectancy (white),Life Expectancy (white) 95% CI - Low,Life Expectancy (white) 95% CI - High,# Deaths,Age-adjusted Death Rate,95% CI - Low.1,95% CI - High.1,Age-Adjusted Mortality (AIAN),Age-Adjusted Mortality (AIAN) 95% CI - Low,Age-Adjusted Mortality (AIAN) 95% CI - High,Age-Adjusted Mortality (Asian),Age-Adjusted Mortality (Asian) 95% CI - Low,Age-Adjusted Mortality (Asian) 95% CI - High,Age-Adjusted Mortality (Black),Age-Adjusted Mortality (Black) 95% CI - Low,Age-Adjusted Mortality (Black) 95% CI - High,Age-Adjusted Mortality (Hispanic),Age-Adjusted Mortality (Hispanic) 95% CI - Low,Age-Adjusted Mortality (Hispanic) 95% CI - High,Age-Adjusted Mortality (white),Age-Adjusted Mortality (white) 95% CI - Low,Age-Adjusted Mortality (white) 95% CI - High,# Deaths.1,Child Mortality Rate,95% CI - Low.2,95% CI - High.2,Child Mortality Rate (AIAN),Child Mortality Rate (AIAN) 95% CI - Low,Child Mortality Rate (AIAN) 95% CI - High,Child Mortality Rate (Asian),Child Mortality Rate (Asian) 95% CI - Low,Child Mortality Rate (Asian) 95% CI - High,Child Mortality Rate (Black),Child Mortality Rate (Black) 95% CI - Low,Child Mortality Rate (Black) 95% CI - High,Child Mortality Rate (Hispanic),Child Mortality Rate (Hispanic) 95% CI - Low,Child Mortality Rate (Hispanic) 95% CI - High,Child Mortality Rate (white),Child Mortality Rate (white) 95% CI - Low,Child Mortality Rate (white) 95% CI - High,# Deaths.2,Infant Mortality Rate,95% CI - Low.3,95% CI - High.3,Infant Mortality Rate (AIAN),Infant Mortality Rate (AIAN) 95% CI - Low,Infant Mortality Rate (AIAN) 95% CI - High,Infant Mortality Rate (Asian),Infant Mortality Rate (Asian) 95% CI - Low,Infant Mortality Rate (Asian) 95% CI - High,Infant Mortality Rate (Black),Infant Mortality Rate (Black) 95% CI - Low,Infant Mortality Rate (Black) 95% CI - High,Infant Mortality Rate (Hispanic),Infant Mortality Rate (Hispanic) 95% CI - Low,Infant Mortality Rate (Hispanic) 95% CI - High,Infant Mortality Rate (white),Infant Mortality Rate (white) 95% CI - Low,Infant Mortality Rate (white) 95% CI - High,% Frequent Physical Distress,95% CI - Low.4,95% CI - High.4,% Frequent Mental Distress,95% CI - Low.5,95% CI - High.5,% Adults with Diabetes,95% CI - Low.6,95% CI - High.6,# HIV Cases,HIV Prevalence Rate,# Food Insecure,% Food Insecure,# Limited Access,% Limited Access to Healthy Foods,# Drug Overdose Deaths,Drug Overdose Mortality Rate,95% CI - Low.7,95% CI - High.7,Drug Overdose Mortality Rate (AIAN),Drug Overdose Mortality Rate (AIAN) 95% CI - Low,Drug Overdose Mortality Rate (AIAN) 95% CI - High,Drug Overdose Mortality Rate (Asian),Drug Overdose Mortality Rate (Asian) 95% CI - Low,Drug Overdose Mortality Rate (Asian) 95% CI - High,Drug Overdose Mortality Rate (Black),Drug Overdose Mortality Rate (Black) 95% CI - Low,Drug Overdose Mortality Rate (Black) 95% CI - High,Drug Overdose Mortality Rate (Hispanic),Drug Overdose Mortality Rate (Hispanic) 95% CI - Low,Drug Overdose Mortality Rate (Hispanic) 95% CI - High,Drug Overdose Mortality Rate (white),Drug Overdose Mortality Rate (white) 95% CI - Low,Drug Overdose Mortality Rate (white) 95% CI - High,# Motor Vehicle Deaths,Motor Vehicle Mortality Rate,95% CI - Low.8,95% CI - High.8,MV Mortality Rate (AIAN),MV Mortality Rate (AIAN) 95% CI - Low,MV Mortality Rate (AIAN) 95% CI - High,MV Mortality Rate (Asian),MV Mortality

We don't want the percentiles or the z-scores and the way the data is formatted they're not tied to their subjects anyway, so we can filter those columns out

In [13]:
#save this in case we want more data later and it becomes easier to take columns away than to simply filter by list
# data.drop(list(data.filter(regex = 'Z-Score')), axis = 1, inplace = True)
# data.drop(list(data.filter(regex = 'CI -')), axis = 1, inplace = True)

In [14]:
#we only need a few columns here
cols = ['NAME', 'Health Outcomes', 'Health Factors', 'Life Expectancy', '% Frequent Physical Distress', '% Frequent Mental Distress']
data = data[cols]

In [15]:
#we're only using rankings
data = data.loc[data['NAME'] != 'Tennessee']

In [16]:
#index appropriately and append to match to full GEOIDs in custom module
data['NAME'] = data['NAME'] + ' County, Tennessee'
data['NAME'] = data['NAME'].str.strip()

In [17]:
cols = list(data.columns)
cols.remove('NAME')
data[cols] = data[cols].astype(float)

In [18]:
#map the common name to the geoid dict in custom module (this only contains the counties in the region currently, but if the module is edited it will contain all)
data['GEO_ID'] = data['NAME'].map(geotogeoid)
#add data source
data['Source'] = 'University of Wisconsin County Health Rankings'

In [19]:
#final check
data.head()

,NAME,Health Outcomes,Health Factors,Life Expectancy,% Frequent Physical Distress,% Frequent Mental Distress,GEO_ID,Source
1,"Anderson County, Tennessee",38.0,14.0,74.660459,15.4,18.2,NaN,University of Wisconsin County Health Rankings
2,"Bedford County, Tennessee",42.0,62.0,74.098790,16.2,18.3,NaN,University of Wisconsin County Health Rankings
3,"Benton County, Tennessee",86.0,80.0,72.033756,18.2,20.5,NaN,University of Wisconsin County Health Rankings
4,"Bledsoe County, Tennessee",21.0,90.0,77.405410,18.0,19.6,NaN,University of Wisconsin County Health Rankings
5,"Blount County, Tennessee",8.0,7.0,76.734427,14.4,17.4,NaN,University of Wisconsin County Health Rankings


In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 1 to 96
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   NAME                          95 non-null     object 
 1   Health Outcomes               95 non-null     float64
 2   Health Factors                95 non-null     float64
 3   Life Expectancy               96 non-null     float64
 4   % Frequent Physical Distress  96 non-null     float64
 5   % Frequent Mental Distress    96 non-null     float64
 6   GEO_ID                        15 non-null     object 
 7   Source                        96 non-null     object 
dtypes: float64(5), object(3)
memory usage: 6.8+ KB


In [21]:
#export to the SQLite database
conn = sq.connect('../../Outputs/Dem_Transpo_Housing_Collection.db')
data.to_sql('UWisconsin_CountyHealthRankings_2022', conn, if_exists = 'replace', index = False)

96